In [1]:
import numpy
import scipy
import math
import numba
import csv
from pyevtk.hl import gridToVTK
from pyevtk.hl import pointsToVTK
import random

In [2]:
N_x = 1501; 
N_y = 1501;
N_points = 400;
manifold = numpy.zeros((N_x,N_y,1));
points = numpy.zeros((N_points,3));

x_0=-2;
y_0=-2;
x_l=2;
y_l=2;
delta_t = 1;
delta_x = (x_l-x_0)/(N_x-1);
delta_y = (y_l-y_0)/(N_y-1);
t = 0;
surface_tilt = 1E5;
t_max = 3000;
t_max_surface = 450;
damping_rate = 1;
mu= 0;
sigma = 0.03;

In [3]:
def init_points(m_points):
    m_points[:,0]=0.5;
    m_points[:,0]=eval_manifold(0.5,0,0);
    return m_points;

In [4]:
def write_vtk_surf(m_manifold,m_t):
    m_manifold[:,2]*=50;
    x = x_0+numpy.linspace(0, (N_x-1)*delta_x, N_x);
    y = y_0+numpy.linspace(0, (N_y-1)*delta_x, N_y);
    z = numpy.linspace(0, delta_x, 1);
    gridToVTK("./results/surface_"+str(m_t), x, y, z, pointData = {"manifold":m_manifold});

In [5]:
# a simple file writing tool for visualisation - vtk is recommended instead
def write_points(t_, samplerate_, u_):
    if (t_ % samplerate_ == 0):   
        with open('./results/points_'+str(t_)+'.csv', 'w', newline='') as csvfile:
            fieldnames = ['x','y','z']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader() 
            for i_ in range(N_points): 
                writer.writerow({'x': str(u_[i_,0]), 'y': str(u_[i_,1]), 'z': str(50.*u_[i_,2])})

In [6]:
@numba.njit(parallel=True, fastmath=True)
def eval_manifold(m_x, m_y, m_t):
    a = [-0.007,-0.009,0.018,0.0261,5];
    b = [1.4282,-0.185,0.8066];
    c = [185,111,74];
    e = [((c[0]*b[0] - c[1]*b[1])*m_x**2 + (c[0]*b[1] - c[1]*b[0])*m_y**2)/(2*(c[0]**2 - c[1]**2)),((c[0]*b[1] - c[1]*b[0])*m_x**2 + (c[0]*b[0] - c[0]*b[0])*m_y**2)/(2*(c[0]**2 - c[1]**2)),(b[2]*m_x*m_y)/c[2]];
    return m_t/surface_tilt*m_x+a[0]/2*(m_x**2 + m_y**2) + a[1]/4*(m_x**4 + m_y**4) + a[2]/2*(m_x**2*m_y**2)+ a[3]/6*(m_x**6 + m_y**6) + a[4]/4*(m_x**4*m_y**4)-(b[0]/2)*(e[0]*m_x**2 + e[1]*m_y**2) - b[1]/2*(e[0]*m_y**2 + e[1]*m_x**2) + 2*b[2]*(e[2])*m_x*m_y;

In [7]:
@numba.njit(parallel=True, fastmath=True)
def calc_manifold(m_manifold,m_t):
    for i in range(N_x):
        for j in range(N_y):
            x = x_0+delta_x*i;
            y = y_0+delta_x*j;
            m_manifold[i,j]=eval_manifold(x,y,m_t);
    return m_manifold;

In [8]:
@numba.njit(parallel=True, fastmath=True)
def update_points(m_points,m_t):
    for i in range(N_points): 
        a = [-0.007,-0.009,0.018,0.0261,5];
        b = [1.4282,-0.185,0.8066];
        c = [185,111,74];
        m_x = m_points[i][0];
        m_y = m_points[i][1];
        e = [((c[0]*b[0] - c[1]*b[1])*m_x**2 + (c[0]*b[1] - c[1]*b[0])*m_y**2)/(2*(c[0]**2 - c[1]**2)),((c[0]*b[1] - c[1]*b[0])*m_x**2 + (c[0]*b[0] - c[0]*b[0])*m_y**2)/(2*(c[0]**2 - c[1]**2)),(b[2]*m_x*m_y)/c[2]];        
        update_x = m_t/surface_tilt+a[0]*(m_x) + a[1]*(m_x**3) + a[2]*(m_x*m_y**2)+ a[3]*(m_x**5) + a[4]*(m_x**3*m_y**4)-b[0]*(e[0]*m_x) - b[1]*(e[1]*m_x) + 2*b[2]*(e[2])*m_y;
        update_y = a[0] *(m_y) + a[1]*(m_y**3) + a[2]*(m_x**2*m_y)+ a[3]*(m_y**5) + a[4]*(m_x**4*m_y**3)-(b[0])*(e[1]*m_y) - b[1]*(e[0]*m_y) + 2*b[2]*(e[2])*m_x;
        
        sx = numpy.random.normal(mu, sigma);
        sy = numpy.random.normal(mu, sigma);
        m_points[i][0] -= damping_rate/delta_t*(update_x+sx);
        m_points[i][1] -= damping_rate/delta_t*(update_y+sy);        
        m_points[i][2] = eval_manifold(m_points[i][0],m_points[i][1],m_t);
    return m_points;

In [9]:
points = init_points(points);
while (t<t_max):
    if (t<t_max_surface):
        calc_manifold(manifold,t);    
        write_vtk_surf(manifold,t);
        update_points(points,t);
    else:
        update_points(points,t_max_surface);
    write_points(t,1,points)
    t+=1;